In [1]:
from apiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import pandas as pd
import os

CLIENT_SECRET_FILE = 'client_secret.json'
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']

flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
credentials = flow.run_console()
youtube = build('youtube', 'v3', credentials=credentials)


#https://www.youtube.com/channel/UCC5hntNsVE7rQRYqqv_ON0g/


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=233696733563.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=nxLFCX1oNN5FkU8LfiT0m1eUepc2KZ&prompt=consent&access_type=offline
Enter the authorization code: 4/zgFQZu_g3MVfr98omjY9QglOq20IWE9ByLk8HJ2KbcY96sWhqbBYZ20


In [2]:
def get_playlist(token):
    request = youtube.playlists().list(
            part="snippet,contentDetails",
            mine=True,
            maxResults=50,
            pageToken=token
        )
    response = request.execute()
    return response
    

def get_list(token,vtype):
    request = youtube.search().list(
        part="snippet",
        forMine=True,
        type=vtype,
        maxResults=50,
        order="date",
        pageToken=token
    )
    response = request.execute()
    #print(response)
    return response

def show_videolist():
    data =pd.DataFrame()
    count =0
    size=50
    max=1
    nextPageToken =None
    while(count*size <= max) :
        rs=get_list(nextPageToken,'video')
        max=int(rs['pageInfo']['totalResults'])
        nextPageToken= rs.get('nextPageToken')
        val=[{'id':d['id']['videoId'], 'title' :d['snippet']['title'] }  for d in rs['items']] 
        print(len(val),max,nextPageToken)
        data=data.append(val)
        count+=1  
    return data

def show_youtube_playlist():
    data =pd.DataFrame()
    count =0
    size=50
    max=1
    nextPageToken =None
    while(count*size <= max) :
        rs=get_playlist(nextPageToken)
        max=int(rs['pageInfo']['totalResults'])
        #print(rs)
        nextPageToken= rs.get('nextPageToken')
        val=[{ 'playlist_id': item['id'], 'playlist_title': item['snippet']['title'] ,'video_count': item["contentDetails"]["itemCount"]} for item in rs['items']]
        data = data.append(val)
        count+=1  
    return data

def create_playlist(title):
    request = youtube.playlists().insert(
        part="snippet,status",
        body={
          "snippet": {
            "title":  title,
            "description":  title,
            "tags": [
              "ML,AI "          
            ],
            "defaultLanguage": "en"
          },
          "status": {
            "privacyStatus": "Unlisted"
          }
        }
    )
    response = request.execute()
    if(response is not None and 'id' in response) :
        print("playlist added successfully with id :{0}".format(response["id"]))
    else:
          print("failed to create playlist ")
          print(response)
        
def add_playlist_item(playlist_id, video_id):
    request = youtube.playlistItems().insert(
        part="snippet",
        body={
          "snippet": {
            "playlistId": playlist_id,
            "resourceId": {
              "kind": "youtube#video",
              "videoId": video_id
            }
          }
        }
    )
    response = request.execute()
    if(response.get('id') is not None) :
        print('item added ' + str(id))
    #print(response)

def delete_playlist(playlist_id):
     request =  youtube.playlists().delete(id=playlist_id)
     response = request.execute()
     print(response)
        

In [3]:
def get_video_files(fullpath):
    filter_extension=".ts"
    data = [ { 'course_type' :path.split('\\')[6].split('(')[0][-1],
               'playlist' :"ML & AI |" + '|'.join(path.split('\\')[6:]) , \
               'file_name':os.path.splitext(file)[0],  "path" : path} \
            for path, subdirs, files in os.walk(fullpath) for file in files if file.endswith(filter_extension)] 
    df= pd.DataFrame(data)
    return df

def create_playlist_data(fullpath, ptype=None):
    df =get_video_files(fullpath)
    if(ptype is not None) :
        df = df[df['course_type']== str(ptype)]     
    groups=df.groupby('playlist')
    return list(groups.groups.keys())



In [ ]:

#add videolist item
#add_playlist_item(youtube,"PL9tQy71sPLeHoB-g1H08J1hR8ZsXWzz7Z","05yT5rR-XsM")



#get_list(None,'video')
#create playlist

#create_playlist(youtube,"")

#delete_playlist()




In [ ]:
##video listing
rs= show_videolist()
rs.to_csv('youtube_videlist.csv', index=False)
len(rs)

In [ ]:
##create video list from dir path
df=get_video_files(r"C:\Users\Ashwini\Downloads\PGDM\Video")
#df.to_csv('dir_file_status.csv', index=False)


In [ ]:
##show playlists
df=show_youtube_playlist()
df.to_csv('youtube_playlist.csv', index=False)

In [11]:
# create playlist by this on youtube 

dt=create_playlist_data(r"C:\Users\Ashwini\Downloads\PGDM\Video",3)

for g in dt[15:]: #remaining
    print(g)
    #create_playlist(g)

print(len(dt))


ML & AI |Course 2(Machine Learning - 1)|M-6 (Additional References)|S-1(Cost Function)
playlist added successfully with id :PLMCuzyPpc7o6SXwKb2oOVDoDiwaXf0vnb
ML & AI |Course 2(Machine Learning - 1)|M-6 (Additional References)|S-2(Gradient Descent-Linear Regression)
playlist added successfully with id :PLMCuzyPpc7o5rCuQVWrNvCs80iVB7ubTw
ML & AI |Course 2(Machine Learning - 1)|M-6 (Additional References)|S-3(Gradient Descent-Advance Regression)
playlist added successfully with id :PLMCuzyPpc7o5justXFX8OYQWuH6jKbd8v
ML & AI |Course 2(Machine Learning - 1)|M-6 (Additional References)|S-4(Logistic Regression)
playlist added successfully with id :PLMCuzyPpc7o46NAnEMT1l9IfEhqlj3Np4
ML & AI |Course 2(Machine Learning - 1)|M-6 (Additional References)|S-5(Application of MLE  Naive Bayes Classifier)
playlist added successfully with id :PLMCuzyPpc7o6_EWpA7tlB_y95R71RpzF6
ML & AI |Course 2(Machine Learning - 1)|M-6 (Additional References)|S-6(Maximum Likelihood Estimation)
playlist added successfu

In [ ]:
##get file to map youtube video with playlist

file_status = pd.read_csv('dir_file_status.csv')
youtube_videlist = pd.read_csv('youtube_videlist.csv')
youtube_playlist= pd.read_csv('youtube_playlist.csv')


del file_status['path']
file_status['file_name']= file_status['file_name'] \
.str.replace('-',' ').str.replace('(','').str.replace(')','').str.replace('.',' ')


print(len(file_status['file_name']),len(youtube_videlist['title']))

rs1=file_status.merge(youtube_videlist,left_on='file_name', right_on='title', how='left')
rs2=rs1.merge(youtube_playlist,left_on='playlist', right_on='playlist_title', how='left')
rs2.to_csv('result.csv', index=False)

#file_status['file_name']==youtube_videlist['title']

#rs1[rs1['id'].isnull()].to_csv('not_uploaded.csv')


In [ ]:
##LINK VIDEO WITH PLAYLIST

rs = pd.read_csv('result.csv')

rs=rs[rs['course_type']==3]

print(len(rs))

rs.sort_values(by=['title'], inplace=True)

#rs= rs[131:]

for i,r in rs.iterrows(): #116
    print(i,r['playlist_id'],r['id'],r['title'])
    #add_playlist_item(r['playlist_id'],r['id'])
    
    
#rs.isnull().sum()
 #115